In [0]:
# loading data, Lazy method
#'/FileStore/tables/wordcount/book.txt'
fileRdd = sc.textFile('/FileStore/tables/wordcount/shakespeare.txt')
print("partitions ", fileRdd.getNumPartitions())

fileRdd = fileRdd.repartition(20) # creates 20 partitions, distribute the data to 20 partitions
print("partitions ", fileRdd.getNumPartitions())


partitions 2
partitions 20

In [0]:
fileRdd.count() # action method, line count

Out[3]: 2469

In [0]:
fileRdd.collect()

Out[3]: ['1. Paradigms of Artificial Intelligence Programming: Case Studies in Common Lisp',
 '2. Code: The Hidden Language of Computer Hardware and Software',
 '3. An introduction to algorithms',
 '4. Artificial Intelligence: a modern approach',
 '5. ON LISP',
 '6. ANSI COMMON LISP',
 '7. LISP IN SMALL PIECES',
 '8. THE LITTLE LISPER',
 '9. THE SEASONED SCHEMER']

In [0]:
lowerCaseRdd = fileRdd.map ( lambda line: line.lower())
# take fetch the data from first partition, first 2 records, better than collect
lowerCaseRdd.take(2) # action method

Out[4]: ["your self again after your self's decease,",
 'when your sweet issue your sweet form should bear. ']

In [0]:
# split the line to list of words
wordsListRdd = lowerCaseRdd.map (lambda line: line.split(" "))
wordsListRdd.take(10) # returns list of list of words

Out[5]: [['your', 'self', 'again', 'after', 'your', "self's", 'decease,'],
 ['when',
 'your',
 'sweet',
 'issue',
 'your',
 'sweet',
 'form',
 'should',
 'bear.',
 '',
 ''],
 ['who', 'lets', 'so', 'fair', 'a', 'house', 'fall', 'to', 'decay,'],
 ['which', 'husbandry', 'in', 'honour', 'might', 'uphold,'],
 ['against', 'the', 'stormy', 'gusts', 'of', "winter's", 'day'],
 ['and', 'barren', 'rage', 'of', "death's", 'eternal', 'cold?'],
 ['o', 'none', 'but', 'unthrifts,', 'dear', 'my', 'love', 'you', 'know,'],
 ['you', 'had', 'a', 'father,', 'let', 'your', 'son', 'say', 'so.'],
 [''],
 ['not', 'from', 'the', 'stars', 'do', 'i', 'my', 'judgement', 'pluck,']]

In [0]:
# convert list of list of words/elements into words list
# flatMap, flatten an array/list into individual element
wordsRdd = wordsListRdd.flatMap(lambda list: list) # converts into words list/flatten
wordsRdd.take(5)

Out[6]: ['your', 'self', 'again', 'after', 'your']

In [0]:
# software: 1 but as tuple (software, 1)
# hardware: 1 but as tuple (hardware, 1)

wordsPairRdd = wordsRdd.map (lambda word: (word, 1) ) # return tuple  (<<word>>, 1) (software, 1)
wordsPairRdd.take(5)

Out[7]: [('your', 1), ('self', 1), ('again', 1), ('after', 1), ('your', 1)]

In [0]:
# word count now, using reduceByKey

"""
 ('paradigms', 1) - paired RDD , key and a value (key = paradigms, value = 1)
 acc = accumulator where the results will be stored, managed by reduceByKey internally
 
 Input: wordsPairRdd
 
 ('artificial', 1),     -- reduceByKJey is not called
  ('in', 1),   -- reduceByKJey is not called
  ('paradigms', 1) -- reduceByKJey is not called
 ('artificial', 1), <- calls reducebyKey, as artifical already there in temp result
                       calls reducebyKey(accomulatorValue, value) => accomulatorValue + value     (1=acc, 1=value ) => 1 + 1 (2)
                       then it updates the  temp result with new value 2
('artificial', 1), <- calls reducebyKey (2 = acc, 1 = value) =>2 + 1 (3)

reduceByKey shall not be called for the first entry 
A temp data structure within reduceByKey

    key,      Accumulator
('artificial',   3)                
 ('in'         , 1)                
('paradigms'    , 1)                    


"""

wordCountRdd = wordsPairRdd.reduceByKey(lambda acc, value: acc + value)
wordCountRdd.take(20)

Out[8]: [('self', 67),
 ('', 559),
 ('of', 368),
 ('none', 7),
 ('till', 14),
 ('head', 2),
 ('no', 71),
 ('was', 28),
 ('have', 72),
 ('would', 21),
 ('leave', 8),
 ("whate'er", 2),
 ('expressing,', 1),
 ('invention', 5),
 ('right,', 4),
 ('expense', 2),
 ('perjured,', 1),
 ("murd'rous,", 1),
 ('despised', 2),
 ('ears', 1)]

In [0]:
# write the result to text file with default partitions
print ("partitions ", wordCountRdd.getNumPartitions())
# prints 2 parition, now we write to text file, 1 partion = 1 task, 2 tasks  total we have.
# this means that, two files shall be created for output based on partition
wordCountRdd.saveAsTextFile('/FileStore/tables/wordcount/results2')

partitions 20

In [0]:
print ("partitions ", wordCountRdd.getNumPartitions())
# prints 2 parition, now we write to text file, 1 partion = 1 task, 2 tasks  total we have.
# this means that, two files shall be created for output based on partition
wordCountRdd = wordCountRdd.coalesce(1)
print ("partitions ", wordCountRdd.getNumPartitions()) # 1
wordCountRdd.saveAsTextFile('/FileStore/tables/wordcount/results3')

partitions 20
partitions 1